In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
from skimage import io,transform

import time

import os
import fnmatch

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True)

def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

def combine_predictions(x,y,y1,y2,thresh,thresh1,thresh2,thresh3):
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    return y_pred

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

In [3]:
x_train = np.zeros((40479,64,64,3), np.float32)
y_train = []

df_train = pd.read_csv('train_label.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (64, 64)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
  
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [11:39<00:00, 82.44it/s]

(40479, 64, 64, 3)
(40479, 17)


In [4]:
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train -= train_mean

In [5]:
#weather classifier (last four labels - mutually exclusive)
x_train, x_val, y_train_w, y_val_w = train_test_split(x_train,y_train[:,-4:],test_size=0.1)
print(x_train.shape)
print(y_train_w.shape)
print(x_val.shape)
print(y_val_w.shape)

(36431, 64, 64, 3)
(36431, 4)
(4048, 64, 64, 3)
(4048, 4)


In [17]:
model = Sequential()#using same architecture for all three models
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
183s - loss: 0.1691 - acc: 0.9358 - val_loss: 0.2742 - val_acc: 0.8926
Epoch 2/10
181s - loss: 0.1352 - acc: 0.9484 - val_loss: 0.1500 - val_acc: 0.9370
Epoch 3/10
181s - loss: 0.1303 - acc: 0.9485 - val_loss: 0.1702 - val_acc: 0.9414
Epoch 4/10
181s - loss: 0.1222 - acc: 0.9519 - val_loss: 0.1777 - val_acc: 0.9332
Epoch 5/10
181s - loss: 0.1167 - acc: 0.9549 - val_loss: 0.1065 - val_acc: 0.9579
Epoch 6/10
181s - loss: 0.1175 - acc: 0.9541 - val_loss: 0.3322 - val_acc: 0.8844
Epoch 7/10
181s - loss: 0.1123 - acc: 0.9553 - val_loss: 0.0997 - val_acc: 0.9583
Epoch 8/10
181s - loss: 0.1089 - acc: 0.9574 - val_loss: 0.0948 - val_acc: 0.9610
Epoch 9/10
181s - loss: 0.1064 - acc: 0.9589 - val_loss: 0.1315 - val_acc: 0.9481
Epoch 10/10
181s - loss: 0.1059 - acc: 0.9593 - val_loss: 0.1179 - val_acc: 0.9539


In [20]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.929144904009
thresh: 0.1 	F2 score: 0.936841238472
thresh: 0.15 	F2 score: 0.935866330228
thresh: 0.2 	F2 score: 0.934888481084
thresh: 0.25 	F2 score: 0.931988754
thresh: 0.3 	F2 score: 0.927547995483
thresh: 0.35 	F2 score: 0.923336627141


In [21]:
from utils import optimise_f2_thresholds
thresh = optimise_f2_thresholds(y_val_w,y_pred, 4)

0 0.36 0.935641351402
1 0.3 0.936323640128
2 0.09 0.939593920572
3 0.11 0.941693722944


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [22]:
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
184s - loss: 0.1003 - acc: 0.9604 - val_loss: 0.0866 - val_acc: 0.9653
Epoch 2/10
182s - loss: 0.0979 - acc: 0.9621 - val_loss: 0.0891 - val_acc: 0.9647
Epoch 3/10
181s - loss: 0.0974 - acc: 0.9624 - val_loss: 0.1053 - val_acc: 0.9573
Epoch 4/10
182s - loss: 0.0969 - acc: 0.9628 - val_loss: 0.0888 - val_acc: 0.9641


In [23]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.9402615048
thresh: 0.1 	F2 score: 0.947810794278
thresh: 0.15 	F2 score: 0.950423783644
thresh: 0.2 	F2 score: 0.951863354037
thresh: 0.25 	F2 score: 0.947551995106
thresh: 0.3 	F2 score: 0.943552371542
thresh: 0.35 	F2 score: 0.93869400527


In [24]:
from utils import optimise_f2_thresholds
thresh = optimise_f2_thresholds(y_val_w,y_pred, 4)

0 0.05 0.953927865613
1 0.15 0.953933747412
2 0.25 0.95431606437
3 0.18 0.954474872953


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [16]:
model.save_weights('weather_1.h5') #95.28 F2
model.save_weights('weather_1_bce.h5') #95.44 F2